In [ ]:
import requests
import json
from azure.storage.blob import BlobServiceClient, ContainerClient

In [ ]:
# Define the base URL and endpoint path for the data
base_url = "https://base_url"
endpoint = "endpoint"

In [ ]:
# Define your authentication credentials as a dictionary
credentials = {
    "tenancyName":"tenancyname",
    "usernameOrEmailAddress":"username",
    "password": "password"
}

In [ ]:
# Initialize Azure Blob Storage client
connection_string = "connection_string"
container_name = "container_name" # For each data create a corresponding folder to store the JSON file

blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [ ]:
# Function to fetch data and store it in Azure Blob Storage
def fetch_and_store_data():
    try:
        # Step 1: Authenticate and obtain a token
        auth_url = base_url + "/api/account/authenticate"
        headers = {"Content-Type": "application/json"}
        auth_response = requests.post(auth_url, headers=headers, json=credentials)
        auth_data = auth_response.json()
        token = auth_data["result"]

        # Step 2: Make a request to the API using the token
        api_url = base_url + endpoint
        headers = {
            "Accept": "application/json",
            "Authorization": f"Bearer {token}"
        }
        response = requests.get(api_url, headers=headers)
        api_data = response.json()

        # Step 3: Extract the data
        if "value" in api_data:
            the_data = api_data["value"]
            
            # Convert the data list to a JSON string
            the_data_json = json.dumps(the_data)

            # Upload the JSON data to the blob
            blob_name = "TheData.json"  # Specify the blob name
            blob_client = container_client.get_blob_client(blob_name)
            blob_client.upload_blob(the_data_json, overwrite=True)
            
            print(f"Data uploaded to '{blob_name}' in '{container_name}' container.")
        else:
            print("No 'value' field found in the API response.")
    except Exception as e:
        print(f"Error: {e}")



In [ ]:
# Fetch and store data
fetch_and_store_data()